# Chapter 32: Deploying to Production

---

Deploying a smart contract to a live network (mainnet) is a significant milestone, but it's also a moment of heightened risk. Unlike testnets, mainnet involves real assets and real consequences. A successful deployment requires careful planning, robust infrastructure, and thorough validation. This chapter covers the entire deployment lifecycle—from preparing your contract for production to post-deployment monitoring and maintenance. You'll learn about deployment strategies, infrastructure choices, automated scripts, and best practices for ensuring a smooth and secure launch.

---

## 32.1 Deployment Strategies

### 32.1.1 Testnet to Mainnet

The standard path is to deploy first to a testnet (e.g., Sepolia, Goerli) before mainnet. Testnets simulate the mainnet environment but use worthless test ETH. This allows you to:

- Verify that your deployment scripts work.
- Test interactions with other contracts (e.g., oracles, bridges) in a live-like environment.
- Allow community members to test your application without risk.
- Verify contract source code on Etherscan (testnet version).

**Checklist before mainnet deployment:**

- [ ] All tests pass on local network and testnet.
- [ ] Contract has been audited (if high value).
- [ ] Deployment address has sufficient funds (ETH for gas).
- [ ] Constructor arguments are correct.
- [ ] Timelocks or multi-sig are set up for privileged roles.
- [ ] Gas estimates are reasonable.
- [ ] Contract verified on testnet Etherscan.
- [ ] Frontend tested against testnet deployment.

### 32.1.2 Gradual Rollout

For high-value protocols, consider a gradual rollout to limit risk:

1. **Soft launch**: Deploy with limited features or caps (e.g., max deposit limit).
2. **Time-locked upgrades**: Use a timelock controller so any changes are delayed, giving users time to react.
3. **Progressive decentralization**: Start with more centralized control (e.g., multi-sig), then gradually transfer to a DAO.
4. **Circuit breakers**: Include pause functionality to halt the contract in case of emergency.

**Example: Launching a lending protocol**
- Week 1: Only whitelisted users can deposit (controlled by admin).
- Week 2: Open to public, but with low deposit caps.
- Week 3: Increase caps, enable borrowing.
- Week 4: Transfer ownership to DAO timelock.

This phased approach allows you to monitor behavior and fix issues before full exposure.

---

## 32.2 Infrastructure Setup

Your DApp needs reliable infrastructure to interact with the blockchain and serve users. This includes RPC nodes, relayers, and monitoring.

### 32.2.1 RPC Node Providers

You cannot rely on users' own nodes (most don't run one). You need RPC endpoints for your frontend and backend to query the blockchain and send transactions.

**Options:**

| Provider | Features | Pricing |
|----------|----------|---------|
| **Infura** | Ethereum, IPFS, popular, free tier | Free up to 100k requests/day |
| **Alchemy** | Enhanced APIs (NFT, Transfers), good analytics | Free tier, pay-as-you-go |
| **QuickNode** | Multi-chain, global nodes | Free tier, then subscription |
| **Moralis** | Web3 APIs, database sync | Free tier, then subscription |
| **Self-hosted** | Run your own node (Geth, Erigon) | Cost of infrastructure |

**Best practices:**
- Use multiple providers for redundancy (fallback).
- Rotate API keys and keep them secret (environment variables).
- Monitor request limits to avoid service interruption.
- For write operations, consider using a dedicated RPC with higher rate limits.

**Example: Fallback provider in ethers.js**
```javascript
const providers = [
  new ethers.providers.JsonRpcProvider('https://eth-mainnet.g.alchemy.com/v2/KEY1'),
  new ethers.providers.JsonRpcProvider('https://mainnet.infura.io/v3/KEY2'),
  new ethers.providers.JsonRpcProvider('https://eth-mainnet.public.blastapi.io')
];

const provider = new ethers.providers.FallbackProvider(providers, 1); // 1 = quorum
```

### 32.2.2 Relayer Services

For gasless transactions or meta-transactions, you may need a relayer. Relayers submit transactions on behalf of users, paying gas in exchange for a fee or signature.

**Options:**
- **OpenZeppelin Defender Relayer**: Managed relay service with autotasks.
- **Gelato Network**: Decentralized relay network.
- **Biconomy**: Relayer infrastructure for meta-transactions.
- **Self-hosted**: Run your own relayer (e.g., using `ethers.js` and a private key).

**Example: Using Defender Relayer**
```javascript
const { DefenderRelayProvider, DefenderRelaySigner } = require('defender-relay-client/lib/ethers');

const credentials = { apiKey: 'YOUR_API_KEY', apiSecret: 'YOUR_API_SECRET' };
const provider = new DefenderRelayProvider(credentials);
const signer = new DefenderRelaySigner(credentials, provider, { speed: 'fast' });

const tx = await signer.sendTransaction({
  to: contractAddress,
  data: contractInterface.encodeFunctionData('functionName', [args])
});
```

### 32.2.3 Monitoring and Alerting

Once live, you need to monitor your contract for unusual activity or failures.

**What to monitor:**
- **Transaction failures**: High revert rates may indicate bugs or attacks.
- **Large transfers**: Unexpected large movements of funds.
- **Oracle updates**: Stale or manipulated prices.
- **Gas prices**: Spikes could indicate MEV attacks.
- **Contract balance**: Should match expectations.

**Tools:**
- **Tenderly**: Transaction monitoring, simulation, alerts.
- **The Graph**: Subgraph for custom event monitoring.
- **OpenZeppelin Defender Sentinel**: Create alerts based on conditions.
- **Custom scripts**: Use web3 libraries to poll and send alerts (Discord, Telegram, PagerDuty).

**Example: Defender Sentinel rule**
- Condition: `event Transfer(address from, address to, uint256 value)`
- Filter: `value > 1000 ether`
- Action: Send Slack message, email.

---

## 32.3 Deployment Scripts

Automated, reproducible deployment scripts are essential for consistency and to avoid human error.

### 32.3.1 Automated Deployments

Use Hardhat or Foundry scripts to deploy your contracts. Scripts should:

- Accept network as a parameter.
- Use environment variables for sensitive data (private keys, API keys).
- Verify contracts automatically.
- Log addresses and transaction hashes.

**Hardhat deploy script example:**
```javascript
// scripts/deploy.js
const { ethers, network, run } = require("hardhat");

async function main() {
  console.log(`Deploying to network: ${network.name}`);

  // Get signers
  const [deployer] = await ethers.getSigners();
  console.log(`Deployer: ${deployer.address}`);

  // Deploy contract
  const Contract = await ethers.getContractFactory("MyContract");
  const contract = await Contract.deploy("constructor arg");
  await contract.waitForDeployment();
  const address = await contract.getAddress();
  console.log(`Contract deployed to: ${address}`);

  // Wait a few blocks for Etherscan to index
  await contract.deploymentTransaction().wait(5);

  // Verify on Etherscan (if not local network)
  if (network.name !== "hardhat" && network.name !== "localhost") {
    try {
      await run("verify:verify", {
        address: address,
        constructorArguments: ["constructor arg"],
      });
      console.log("Contract verified on Etherscan");
    } catch (error) {
      console.error("Verification failed:", error);
    }
  }
}

main().catch(console.error);
```

### 32.3.2 Multi-Network Deployment

Manage different configurations per network (addresses, parameters) using a config file.

**Example: `deploy-config.js`**
```javascript
module.exports = {
  sepolia: {
    uniswapRouter: "0x...",
    initialSupply: ethers.parseEther("1000000"),
    owner: "0x...",
  },
  mainnet: {
    uniswapRouter: "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", // actual Uniswap router
    initialSupply: ethers.parseEther("10000000"),
    owner: "0x...",
  }
};
```

In your script, load the config based on `network.name`.

### 32.3.3 Contract Verification

Verifying source code on Etherscan (or other block explorers) is crucial for transparency. Users can see that the deployed bytecode matches your verified source.

**Manual verification:** Upload source via Etherscan UI.

**Automated verification:** Use Hardhat's `verify` task or `truffle-plugin-verify`.

**Hardhat verify command:**
```bash
npx hardhat verify --network sepolia DEPLOYED_ADDRESS "constructor arg1" "arg2"
```

**In a script:** Use `run("verify:verify", {...})` as shown above.

**Note:** You must wait a few blocks after deployment for Etherscan to detect the contract.

---

## 32.4 Post-Deployment

After deployment, there are critical steps to ensure the contract is properly configured and documented.

### 32.4.1 Ownership Transfer

If your contract uses an owner role (e.g., `Ownable`), you should transfer ownership to a more secure entity:

- **Multi-sig wallet** (e.g., Gnosis Safe) for team-controlled projects.
- **Timelock controller** for DAO-governed projects.
- **Burn the owner key** if no further changes are needed (for immutable contracts).

**Example: Transferring ownership to a Gnosis Safe**
```javascript
const safeAddress = "0x..."; // multi-sig address
await contract.transferOwnership(safeAddress);
```

**Why?** A single private key is a single point of failure. If compromised, the attacker could upgrade or destroy the contract. A multi-sig requires multiple signatures, greatly reducing risk.

### 32.4.2 Initial Configuration

Some contracts require post-deployment setup:

- Setting up oracles (e.g., Chainlink price feeds).
- Adding initial liquidity to a DEX pool.
- Whitelisting certain addresses.
- Pausing/unpausing (if started paused).

**Example: Setting price feed after deployment**
```javascript
await contract.setPriceFeed(chainlinkEthUsdAddress);
```

**Checklist:**
- [ ] All required roles are granted.
- [ ] External dependencies (oracles, bridges) are correctly configured.
- [ ] Initial parameters are set.
- [ ] Contract is unpaused if needed.

### 32.4.3 Documentation

Provide clear documentation for users and developers:

- **Contract address** on mainnet (and testnets).
- **ABI** and source code (Etherscan link).
- **Interface description**: which functions to call, events to listen for.
- **Admin roles**: who has what permissions.
- **Upgradeability**: if contract is upgradeable, explain the mechanism.
- **Security contact**: where to report vulnerabilities.

**Publish on:**
- Project website.
- GitHub repository (README).
- Developer docs (e.g., GitBook, ReadTheDocs).

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 32 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Production deployment requires careful planning and            │
│  infrastructure.                                                │
│                                                                 │
│  Deployment strategies:                                         │
│    • Testnet first, then mainnet                               │
│    • Gradual rollout with caps and timelocks                   │
│                                                                 │
│  Infrastructure:                                                │
│    • RPC providers (Infura, Alchemy, QuickNode)                │
│    • Relayers for gasless transactions                         │
│    • Monitoring (Tenderly, Defender Sentinels)                 │
│                                                                 │
│  Deployment scripts:                                            │
│    • Automate with Hardhat/Foundry                             │
│    • Support multiple networks via config                      │
│    • Auto-verify on Etherscan                                  │
│                                                                 │
│  Post-deployment:                                               │
│    • Transfer ownership to multi-sig or timelock               │
│    • Perform initial configuration                             │
│    • Document everything (addresses, roles, security contact)  │
│                                                                 │
│  A successful launch is just the beginning—monitor and         │
│  maintain your contract.                                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 33 – Monitoring and Maintenance. We'll dive deeper into monitoring strategies, incident response, and ongoing maintenance of live contracts.